# 用torchvision自带的官方图片数据集

参考代码：https://blog.csdn.net/qq_34714751/article/details/85610966

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import mnist
import torch.nn.functional as F
import torch.optim as optim

读入数据增强后的，这里做了归一化，均值0.5，方差0.5

In [2]:
trans=transforms.Compose([transforms.ToTensor(),
                          transforms.Normalize([0.5],[0.5])]
)

这里后来查了下，原来 transforms.ToTensor() 是除以 255 且转 tensor
然后 transforms.Normalize -0.5/0.5 归一化到了 [-1,1]

In [3]:
# num_workers 默认 = 0，我们知道加载 batch 是到 RAM 中找，所以 batch 大小很有讲究，这样就可以避免频繁缺块
# 但是注意我们这样还是得找，如果指定 worker 来负责读写的这个任务就会快一些
# num_workers 可设置的大小取决于 CPU 逻辑核心数
train_set=mnist.MNIST('official_data',train=True,transform=trans)
test_set=mnist.MNIST('official_data',train=False,transform=trans)
train_loader=DataLoader(train_set,batch_size=64,shuffle=True,num_workers=4)
test_loader=DataLoader(test_set,batch_size=64,shuffle=True,num_workers=4)

In [4]:
train_set

Dataset MNIST
    Number of datapoints: 60000
    Root location: official_data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )

In [5]:
train_loader.batch_size

64

对一个迭代类型，当你不知道怎么读时
一定可以用 enumerate 或者 iter 暴力读出，参考代码：
for i,xy in enumerate(train):
    x,y=xy
很神奇的是 tensor 没有 ndarray, tuple 之类的类型，全是混在一起的，但是可以用那样的用法，就像是潜在的类型
比如上面可能是一个数字5
可能一个样本的数据是这样(注意一个样本，丢了batch维，然后是单通道维，然后是行维、列维)
维度 B、C、H、W，注意 B、C 所处的位置
tensor(\[\[\[1,2\],\[2,3\]\]\],5)

LeNet-5架构(无修正版)：当然还是有一点修正，运行时发现这个数据和吴恩达说好的不一样，怎么是 28\*28 的，不是32\*32，于是我在下一种写法里加了 padding

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        # 注意，也不要忘了偏置单元的事，一般默认都是 bias=True，所以没事
        self.conv1=nn.Sequential(
            # nn.Conv2d(in_channels=1,
            #           out_channels=6,
            #           kernel_size=5),
            nn.Conv2d(1,6,5), # 默认参数 in_channels,out_channels,kernel_size,stride=1,padding=0
            nn.ReLU(),
            # nn.AvgPool2d(kernel_size=2,
            #              stride=2)
            nn.AvgPool2d(2,2) # 默认参数 kernel_size,stride=0,padding=0
        )
        self.conv2=nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.AvgPool2d(2,2)
        )
        self.fc1=nn.Sequential(
            nn.Linear(400,120), # in_features, out_features 输出特征数就是本层结点数
            nn.ReLU()
        )
        self.fc2=nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU()
        )
        self.out=nn.Linear(84,1)
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        # -1 表示自动计算，比如总共 8，-1*4 就是 2*4 的意思，这里 -1 实际上就是 batch size
        x=x.view(-1,self.num_flat_features(x))
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.out(x)
        return x
    def num_flat_features(self, x): # 这个 flatten 就和模板一样，以后用连改都不用改
        size=x.size()[1:]  # m*5*5*16，第一维是 batch size，每个样本 flatten 成 5*5*16=400 的列向量
        num_features=1
        for s in size:
            num_features*=s
        return num_features
cnn=CNN()
cnn

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (conv2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (fc1): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
  )
  (fc2): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
  )
  (out): Linear(in_features=84, out_features=1, bias=True)
)

或者，下面这种写法更常用一些，一些fuctional写在前向传播里

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(1,6,5,padding=2)
        self.conv2=nn.Conv2d(6,16,5)
        self.pool=nn.AvgPool2d(2,2)
        self.fc1=nn.Linear(400,120)
        self.fc2=nn.Linear(120,84)
        self.out=nn.Linear(84,1)
    def forward(self,x):
        # 这也是一种写法
        # x=F.avg_pool2d(F.relu(self.conv1(x)),2) # F的pool默认参数有点区别，这里是默认 stride=kernel_size
        # x=F.avg_pool2d(F.relu(self.conv2(x)),2)
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(-1,self.num_flat_features(x)) # 听说 torch 的 reshape 比 view 功能更多，没具体了解，以后还是写reshape吧
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.out(x)
        return x
    def num_flat_features(self, x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features*=s
        return num_features
cnn=CNN()
cnn

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (out): Linear(in_features=84, out_features=1, bias=True)
)

将模型和张量移动到GPU，张量的移动写在下面的梯度下降函数里
我只有一个英伟达的，另一个用不了CUDA，所以这里没用 nn.DataParallel 多 GPUs
（讲真，GPU实在太快了，这一个GPU都比我的四核八线程CPU强好多）

In [8]:
device=torch.device('cuda:0')
cnn=cnn.to(device)

定义损失函数
这个古董版本输出层用的是 RELU，而非 Softmax
第一次写DL代码，令我感到震惊的是，你写逻辑回归的交叉熵损失函数CrossEntropyLoss()它也能用
牛头对马嘴他也能跑，但是逻辑说不通，结果的话训练得多了也不会很差。。。

In [9]:
criterion=nn.MSELoss()
optimizer=optim.Adam(cnn.parameters()) # 按照吴恩达的建议，Adam优化器超参数直接默认就完事

In [10]:
for epoch in range(100):
    # 遍历 train_loader，其实是在遍历 batch，这里我们 batch_size=64 于是 60000/64=938 个 batch
    # 不用 mini-batch 的话就是 1 个 epoch 里用 1 个 batch
    cost=0.0
    for i,train in enumerate(train_loader):
        X_train,y_train=train
        X_train=X_train.to(device) # 写成 X_train.cuda() 就是默认仅放在 gpu:0 上
        y_train=y_train.to(device)
        y_train=y_train.to(torch.float32) # 离谱，torch 连 long 和 float 自动转换都做不到，原始标签是 long
        y_train=y_train.view(y_train.size()[0],1) # 维度也得自己改改，离谱啊
        optimizer.zero_grad() # 初始化梯度为 0，因为是链式求导，会累乘，而我们不要上一个 batch 的
        y_pred=cnn(X_train)
        loss=criterion(y_pred,y_train) # 注意线性激活算MSE，两个都得是float类型
        loss.backward() # 反向传播计算梯度
        optimizer.step() # 用梯度更新
        cost+=loss.item()
    print('[%d] cost: %.3f'%(epoch+1,cost))
print('Finished Training')

[1] cost: 2609.389
[2] cost: 715.124
[3] cost: 475.607
[4] cost: 380.799
[5] cost: 321.586
[6] cost: 275.797
[7] cost: 241.117
[8] cost: 212.992
[9] cost: 188.990
[10] cost: 171.879
[11] cost: 155.069
[12] cost: 144.302
[13] cost: 127.231
[14] cost: 118.793
[15] cost: 119.160
[16] cost: 103.842
[17] cost: 96.602
[18] cost: 93.058
[19] cost: 82.451
[20] cost: 82.932
[21] cost: 73.408
[22] cost: 65.177
[23] cost: 63.881
[24] cost: 65.848
[25] cost: 60.186
[26] cost: 51.232
[27] cost: 52.346
[28] cost: 48.810
[29] cost: 47.064
[30] cost: 42.029
[31] cost: 43.261
[32] cost: 39.202
[33] cost: 40.610
[34] cost: 37.024
[35] cost: 40.194
[36] cost: 31.233
[37] cost: 27.847
[38] cost: 27.205
[39] cost: 42.675
[40] cost: 30.602
[41] cost: 26.491
[42] cost: 22.011
[43] cost: 27.917
[44] cost: 31.764
[45] cost: 28.367
[46] cost: 20.938
[47] cost: 20.407
[48] cost: 24.735
[49] cost: 27.552
[50] cost: 17.990
[51] cost: 24.275
[52] cost: 15.903
[53] cost: 32.545
[54] cost: 20.132
[55] cost: 15.042
[5

In [11]:
y=train_loader.dataset[0][1]
y=torch.tensor(data=y,dtype=torch.float32)
y

tensor(5.)

In [12]:
train_loader.dataset[0][0].shape

torch.Size([1, 28, 28])

In [13]:
for train in iter(train_loader):
    print(train[0].shape)
    break

torch.Size([64, 1, 28, 28])


如上所示
dataset[i] 可以取出(X,y)，而且是严格的一条数据，当然注意掉维问题
我们知道如果你迭代 train_loader 的话取出的是 batch_size 个合并的一条
还要注意的是 DataLoader 的 batch_size 参数默认值是 1，而非样本点个数
这意味着默认的是 SGD 随机梯度下降，而非 batch，更不是 mini-batch

In [14]:
x=train_loader.dataset[0][0]
x=x.view(-1,x.size()[0],x.size()[1],x.size()[2])
x.shape

torch.Size([1, 1, 28, 28])

In [15]:
x=x.to(device)
y_pred=cnn(x)

In [16]:
y_pred

tensor([[5.0036]], device='cuda:0', grad_fn=<AddmmBackward0>)

这里我们保存一下训练好的网络，避免重复训练

In [18]:
torch.save(cnn.state_dict(),'saves/official_cnn_params.pth')

另一种保存方式是直接 torch.save(cnn,路径)
然后这个就是保存整个 cnn 对象的内容，就很慢很大了
我们上面的写法是仅保存了参数
当然保存参数缺点是你还要重新创建对象，设置一些属性之类的
实际上 torch 的 save 本质上应该和 python 的 pickle 或者 shelve 是一样的
就是把对象内容保存为一个二进制文件
然后关于后缀，其实是啥都无所谓，因为不对应任何直接打开的格式
习惯性使用 .pt .pth 或者 .pkl